## Imports

In [25]:
#import libraries 

import numpy as np
import pandas as pd
import datetime
import re
import string
from textblob import TextBlob
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.metrics.pairwise import cosine_similarity

from nltk.tokenize import word_tokenize, MWETokenizer # multi-word expression
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.chunk import ne_chunk
from nltk.tag import pos_tag

from gensim import corpora, models, similarities, matutils

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def date_filter(df, start, end):
    mask = (df.timestamp > pd.to_datetime(start)) & (df.timestamp < pd.to_datetime(end))
    df = df[mask]
    return df

In [8]:
def filter_columns(df, col_list):
    df = df[col_list]
    return df

In [9]:
def filter_by_rating(df, min_stars, max_stars):
    mask = (df.Rating >= min_stars) & (df.Rating <= max_stars)
    df = df[mask]
    return df

In [15]:
#Read the dataset and assigning new column name to the 
review_data = pd.read_csv("data-Project5/books.csv", usecols=['review_id', 'review_date', 'review_body'], index_col='review_id', parse_dates=True)

In [17]:
#Display the dataset
review_data = review_data.rename(columns = {'review_date':'timestamp'})
review_data.head()

,review_body,timestamp
review_id,,
RJOVP071AVAJO,I love it and so does my students!,2015-08-31
R1ORGBETCDW3AI,My wife and I ordered 2 books and gave them as...,2015-08-31
R7TNRFQAOUTX5,Great book just like all the others in the ser...,2015-08-31
R2GANXKDIFZ6OI,So beautiful,2015-08-31
R2NYB6C3R8LVN6,Enjoyed the author's story and his quilts are ...,2015-08-31


In [18]:
review_data.timestamp = pd.to_datetime(review_data.timestamp)
print(max(review_data.timestamp))
print(min(review_data.timestamp).date())

2015-08-31 00:00:00
2012-05-03


In [19]:
df_2012 = date_filter(review_data, '2012, 1, 1', '2012, 12, 31').sort_values('timestamp').reset_index(drop=True)
# df_2013 = date_filter(review_data, '2013, 1, 1', '2013, 12, 31').sort_values('timestamp')
# df_2014 = date_filter(review_data, '2014, 1, 1', '2014, 12, 31').sort_values('timestamp')
# df_2015 = date_filter(review_data, '2015, 1, 1', '2015, 12, 31').sort_values('timestamp')

display(df_2012.head())
display(df_2012.shape)

,review_body,timestamp
0,This is a very inspirational read from a man w...,2012-05-03
1,While the book is very informative on issues o...,2012-05-03
2,I was given this book by a customer who is a l...,2012-05-03
3,Journalist Leo Desroches is going undercover a...,2012-05-03
4,I made thru the first sex scene and then wishe...,2012-05-03


(1122313, 2)

## Text Pre-Processing

In [21]:
additional_stop_words = [
    'like', 'dont', 'im', 'say', 'did', 'said', 'thats', 'don', 'hes', 'does', 'thing', 'gt', 'sure', 'doesnt',
    'saying', 'youre', 'isnt', 'doing', 'got', 'didnt', 'yeah', 'just', 'yes',
    'right', 'think', 'going', 'want', 'know', 'good',
    'need', 'time', 'point', 'make', 'way', 'really',
    'id', 'ar', 's', 't', 've', 'm', 'shes', 
    'c', 'd', 'v', 'actually', 'look', 'maybe', 'though', 'bad', 'came', 'mods', 'things', 'lot', 'let', 'lol', 'tell', 'pretty', 'literally'
    'theyre', 'people',
    '‘', '’', '“'
]
multi_words = [
    ('health','insurance'),
    ('fox', 'news'),
    ('bernie', 'sanders'),
    ('hillary', 'clinton'),
    ('barack', 'obama'),
    ('donald', 'trump'),
    ('joe', 'biden'),
    ('joseph', 'biden'),
    ('mass', 'shooting'),
    ('mass', 'shootings'),
    ('assault', 'weapon'),
    ('assault', 'weapons'),
    ('assault', 'weapons', 'ban'),
    ('sergeant', 'at', 'arms'),
    ('stop', 'and', 'frisk'),
    ('medicare', 'for', 'all'),
    ('public', 'option'),
    ('beat', 'trump'),
    ('articles', 'of', 'impeachment'),
    ('new', 'york'),
    ('hold', 'in', 'contempt'),
    ('quid', 'pro', 'quo')
]
mwe_tokenizer = MWETokenizer(multi_words)
stop_words = ENGLISH_STOP_WORDS.union(additional_stop_words)

In [26]:
X = df_2012.review_body
X_train, X_test = train_test_split(X, test_size=0.25, random_state=42)

In [27]:
cv = CountVectorizer(ngram_range=(1,2), binary=False, stop_words='english', 
                      lowercase=True, strip_accents='unicode', max_df=0.5)

data_cv = cv.fit_transform(X_train)
data_dtm_raw = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm_raw.index = data_clean.index

MemoryError: Unable to allocate 116. TiB for an array with shape (841734, 18939766) and data type int64

In [ ]:
data_dtm_raw.iloc[:5, -171:-150]